In [1]:
import numpy as np
import os

In [6]:
out = open("/code/CMRG_ext_GRCh38.bed.roi", "w")
with open("/code/CMRG_ext_GRCh38.bed") as f:
    for r in f:
        r = r.strip().split()
        print(r[3], "hg38_tagged.fa", "chr"+r[0]+"_hg38", r[1], r[2], 0, sep="\t", file=out)
out.close()

In [7]:
%%bash
mkdir -p /wd/results/cmrg-out
mkdir -p /wd/results/cmrg-out/svg

In [8]:
%%bash
\time -v /wd/code/pgr-fetch-seqs /wd/data/pgr-tk-HGRP-y1-evaluation-set-v0 \
-r /code/CMRG_ext_GRCh38.bed.roi > /wd/results/cmrg-out/ROI_seq.fa

	Command being timed: "/wd/code/pgr-fetch-seqs /wd/data/pgr-tk-HGRP-y1-evaluation-set-v0 -r /code/CMRG_ext_GRCh38.bed.roi"
	User time (seconds): 27.33
	System time (seconds): 423.20
	Percent of CPU this job got: 1224%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:36.79
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 37368008
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 11107218
	Voluntary context switches: 1088574
	Involuntary context switches: 721
	Swaps: 0
	File system inputs: 0
	File system outputs: 43368
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


In [9]:
%%bash
\time -v /wd/code/pgr-query /wd/data/pgr-tk-HGRP-y1-evaluation-set-v0 \
/wd/results/cmrg-out/ROI_seq.fa /wd/results/cmrg-out/pg_seqs --merge-range-tol 100000

	Command being timed: "/wd/code/pgr-query /wd/data/pgr-tk-HGRP-y1-evaluation-set-v0 /wd/results/cmrg-out/ROI_seq.fa /wd/results/cmrg-out/pg_seqs --merge-range-tol 100000"
	User time (seconds): 128.44
	System time (seconds): 327.66
	Percent of CPU this job got: 311%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 2:26.28
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 38025412
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 15685631
	Voluntary context switches: 1648414
	Involuntary context switches: 61036
	Swaps: 0
	File system inputs: 0
	File system outputs: 5183080
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


In [10]:
target_seqs = {}

with open("/wd/results/cmrg-out/pg_seqs.hit") as f:
    for row in f:
        row = row.strip().split("\t")
        if row[0][0] == "#":
            continue
        (q_idx, q_name, query_bgn, query_end, q_len, aln_anchor_count, 
        src, ctg, ctg_bgn, ctg_end, orientation, out_seq_name) = row
        query_bgn = int(query_bgn)
        query_end = int(query_end)
        ctg_bgn = int(ctg_bgn)
        ctg_end = int(ctg_end)
        q_len = int(q_len)
        if query_bgn > 12000 or q_len - query_end > 12000:
            continue
        if abs(ctg_end-ctg_bgn) > q_len * 3:
            continue
        target_seqs.setdefault( (q_idx, q_name), [])
        target_seqs[(q_idx, q_name)].append( (out_seq_name, abs(ctg_end - ctg_bgn)) )

In [ ]:
for q_idx, q_name in target_seqs:

    total_len = 0
    c = 0
    include_file = f"/wd/results/cmrg-out/{q_name}_included_ctgs.{q_idx}"
    with open(include_file, "w") as f:
        for out_seq_name, l in target_seqs[(q_idx, q_name)]:
            c += 1
            print(out_seq_name, file=f)

    lengths = [_[1] for _ in target_seqs[(q_idx, q_name)]]


    fasta_file = f"/wd/results/cmrg-out/pg_seqs.{q_idx}.fa"
    prefix = f"{q_name}_pg_seqs.{q_idx}."

    r = int( 2 * (np.mean(lengths)/50000)**0.5)
    if r < 2:
        r = 2
    if r > 12:
        r = 12
    print(q_idx, q_name, len(lengths), np.mean(lengths), min(lengths), max(lengths), r)
    os.system(f"""/wd/code/pgr-pbundle-decomp -w 48 -k 56 -r {r} \
              --min-span 12 --bundle-length-cutoff 100 --min-branch-size 8 \
              {fasta_file} /wd/results/cmrg-out/{prefix} \
              --min-cov 0 --include {include_file}""")
    bed_file = f"/wd/results/cmrg-out/{prefix}bed"
    os.system(f"/wd/code/pgr-pbundle-bed2sorted {bed_file} /wd/results/cmrg-out/{prefix}")
    ord_file = f"/wd/results/cmrg-out/{prefix}ord"
    os.system("cat " + ord_file + " | awk '{print $1\"\\t\"$1}' > " + ord_file + "2")
    track_range = 10000*(int(max(lengths) / 10000) + 1)
    # print(bed_file)
    os.system(f"""/wd/code/pgr-pbundle-bed2svg {bed_file} /wd/results/cmrg-out/svg/{prefix} \
              --track-range {track_range} --track-tick-interval 10000 \
              --track-panel-width 1200 \
              --stroke-width 1.2  --annotations /wd/results/cmrg-out/{prefix}ord2""")


    os.system(f"/wd/code/pgr-pbundle-bed2dist {bed_file} /wd/results/cmrg-out/{prefix}")
    os.system(f"""/wd/code/pgr-pbundle-bed2svg {bed_file} /wd/results/cmrg-out/svg/{prefix}hc. \
              --track-range {track_range} --track-tick-interval 10000 \
              --track-panel-width 1200 --stroke-width 0.5 \
              --annotations /wd/results/cmrg-out/{prefix}ord2 \
              --ddg-file /wd/results/cmrg-out/{prefix}ddg \
              --highlight-repeats 3""")

000 EXTL2 97 21954.907216494845 21356 22110 2
001 AGRN 97 34632.85567010309 34038 35708 2
002 GSTM1 87 17632.344827586207 7007 45565 2
003 MASP2 97 19399.41237113402 18968 19836 2
004 LRIG2 97 58244.52577319588 57065 58945 2
005 VANGL1 97 53778.969072164946 48334 55856 2
006 NPPA 71 1733.5915492957747 311 2185 2
007 NOTCH2 98 188974.97959183675 188019 194999 3
